In [8]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, f1_score, precision_score
# from tensorflow.keras.callbacks import LearningRateScheduler
# from tensorflow.keras.utils import to_categorical  # Import to_categorical from keras.utils
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense 

# from tensorflow.keras.regularizers import l1_l2,l1,l2
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import StandardScaler
import itertools
import random

In [9]:
# Define a simple neural network model
def create_model(inp_shape,num_classes):
    model = Sequential()
    model.add(Dense(64, activation='leaky_relu', input_shape=inp_shape))
    model.add(Dense(32, activation='leaky_relu'))
    model.add(Dense(num_classes, activation='softmax'))
    return model

# Define the fitness function
def fitness_function(model, X_train, y_train, X_test, y_test, LR):
    model.compile(loss='sparse_categorical_crossentropy', optimizer=Adam(learning_rate=LR), metrics=['accuracy'])
    model.fit(X_train, y_train, epochs=10, verbose=0)
    y_pred = model.predict(X_test)
    accuracy = accuracy_score(y_test, np.argmax(y_pred, axis=1))
    return accuracy

In [10]:
#for reading dataset-1
# Load your CSV data1 using pandas
data1 = pd.read_csv('classData.csv')

# Combine the values of the first 4 columns into a new column
data1['Combination'] = data1.apply(lambda row: ''.join(map(str, row[:4])), axis=1)

# Find unique combinations and assign label numbers
unique_combinations = data1['Combination'].unique()
label_mapping = {combination: label for label, combination in enumerate(unique_combinations)}

# Add a new column with the label numbers
data1['Label'] = data1['Combination'].map(label_mapping)

X1 = data1.iloc[:, 4:10].values  # Features
y1 = data1.iloc[:, 11:].values  # Labels

# Standardize the data
scaler = StandardScaler()
X1 = scaler.fit_transform(X1)

# One-hot encode the labels
# y1_encoded = to_categorical(y1, num_classes=6)
num_classes=6
y1_encoded = y1

In [11]:
#Compiling and testing model for dataset-1

#Splitting dataset-1 into train and test
X_train1, X_test1, y_train1, y_test1 = train_test_split(X1, y1_encoded, test_size=0.2, random_state=42)

input_shape1=(X_train1.shape[1],)
num_classes=6
# model1=configure_model(input_shape1,num_classes)


In [12]:
# Genetic Algorithm Configuration
generations = 10
mutation_rate = 0.1

# Initialize a population of neural network models with random weights
population_size = 10
population = [create_model(input_shape1,num_classes) for _ in range(population_size)]

In [13]:
def generate_population(population,X_train,y_train,X_test,y_test,num_classes,input_shape,LR):
    for generation in range(generations):
        # Evaluate the fitness of each model in the population
        fitness_scores = [fitness_function(model, X_train, y_train,X_test,y_test,LR) for model in population]
        
        # Select the top-performing models (elitism)
        num_parents = 3
        parents = np.argsort(fitness_scores)[-num_parents:]
        
        # Crossover (create new models from the selected parents)
        new_population = [create_model(input_shape,num_classes) for _ in range(population_size)]
        for i in range(num_parents, population_size):
            parent1 = random.choice(parents)
            parent2 = random.choice(parents)
            child_weights = []

            # Iterate through the layers and set the weights individually
            for layer in range(len(population[parent1].get_weights())):
                # Perform crossover by taking the mean of the corresponding weights
                layer_weights = np.mean([population[parent1].get_weights()[layer], population[parent2].get_weights()[layer]], axis=0)
                child_weights.append(layer_weights)

            # Set the weights of the child model
            new_population[i].set_weights(child_weights)

    population = new_population

    return population,fitness_scores



In [14]:

final_population1,fitness_scores1=generate_population(population,X_train1,y_train1,X_test1,y_test1,num_classes,input_shape1,0.0065)

# Find the best model from the final generation
best_model1 = final_population1[np.argmax(fitness_scores1)]
best_accuracy1 = fitness_scores1[np.argmax(fitness_scores1)]
print(f"Best accuracy: {best_accuracy1}")

50/50 [==============================] - 0s 638us/step
Best accuracy: 0.8785759694850604


In [15]:
#Dataset preprocessing for dataset-2

# Load your CSV data using pandas
data2 = pd.read_csv('cwru_bearing.csv')
import pandas as pd

# Step 1: Identify unique labels in the last column
unique_labels = data2['fault'].unique()

# Step 2: Create a mapping from labels to numbers
label_to_number = {label: index for index, label in enumerate(unique_labels)}

# Step 3: Replace labels with corresponding numbers
data2['fault'] = data2['fault'].map(label_to_number)

# Now, 'Y' contains numeric labels from 0-9

# You can save the modified DataFrame to a new CSV file if needed
# data2.to_csv('cwru_bearing_numeric_labels.csv', index=False)
# Standardize the data

X2 = data2.iloc[:, 0:9].values  # Features
y2 = data2.iloc[:, 9].values  # Labels

# Standardize the data
scaler = StandardScaler()
X2 = scaler.fit_transform(X2)


In [16]:
#Compiling and testing model for dataset-2

#Splitting dataset-1 into train and test
X_train2, X_test2, y_train2, y_test2 = train_test_split(X2, y2, test_size=0.2, random_state=42)
num_classes=10
input_shape2=(X_train2.shape[1],)

# model2=configure_model(input_shape2,num_classes)

In [17]:
# Genetic Algorithm Configuration
generations = 10
mutation_rate = 0.1

# Initialize a population of neural network models with random weights
population_size = 10
population = [create_model(input_shape2,num_classes) for _ in range(population_size)]

In [18]:
final_population2,fitness_scores2=generate_population(population,X_train2,y_train2,X_test2,y_test2,num_classes,input_shape2,0.0169)

# Find the best model from the final generation
best_model2 = final_population2[np.argmax(fitness_scores2)]
best_accuracy2 = fitness_scores2[np.argmax(fitness_scores2)]
print(f"Best accuracy: {best_accuracy2}")

15/15 [==============================] - 0s 1ms/step
Best accuracy: 0.9652173913043478


In [19]:
print(best_model1.summary())
print(best_model2.summary())

Model: "sequential_176"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_528 (Dense)           (None, 64)                448       
                                                                 
 dense_529 (Dense)           (None, 32)                2080      
                                                                 
 dense_530 (Dense)           (None, 6)                 198       
                                                                 
Total params: 2726 (10.65 KB)
Trainable params: 2726 (10.65 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None
Model: "sequential_284"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_852 (Dense)           (None, 64)                640       
                                                       

In [21]:
# Save the model architecture to a JSON file
# model_json = best_model1.to_json()
# with open("model_architecture.json", "w") as json_file:
#     json_file.write(model_json)

# Save the model weights to an HDF5 file
# best_model1.save_weights("model_weights1.h5")
best_model1
best_model1.save("model1.keras")
best_model2.save("model2.keras")
# # Save the model architecture to a JSON file
# model_json = best_model2.to_json()
# with open("model_architecture.json", "w") as json_file:
#     json_file.write(model_json)

# # Save the model weights to an HDF5 file
# best_model2.save_weights("model_weights2.h5")
